In [1]:
import random
import re
import math
import glob

import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from torchinfo import summary
from gensim.models import Word2Vec
from tqdm import tqdm

In [2]:
class TextTrainDataset(IterableDataset):
    
    def __init__(self, dataset_path, wv, seq_length=10, target_length=1, padding_factor=20, padding_limit=3, epoch_size=100_000):
        self.dataset_path = dataset_path
        self.wv = wv
        self.seq_length = seq_length
        self.target_length = target_length
        self.padding_factor = padding_factor
        self.padding_limit = padding_limit
        self.epoch_size = epoch_size
        
        self.paths = list(glob.glob(f'{dataset_path}/**/*.txt', recursive=True))
        self.distribution = self.__construct_distribution()
        
    def __len__(self):
        return np.sum(self.distribution)
        
    def __iter__(self):
        distribution = np.array(self.distribution)
        indexes = np.arange(len(self.paths))
        
        while distribution.any():
            text_index = np.random.choice(indexes, p=distribution/distribution.sum())
            distribution[text_index] -= 1
            text = self.__get_text(self.paths[text_index])
            yield self.__get_sample_from_text(text)
    
    def __get_text(self, path):
        with open(path) as f:
            text = f.read()
            return text.split(' ')[:-1]
    
    def __construct_distribution(self):
        lengths = self.__get_files_lengths()
        distribution = np.ceil(lengths / lengths.sum() * self.epoch_size).astype(np.int32)
        return distribution
    
    def __get_files_lengths(self):
        lengths = []
        for path in self.paths:
            text = self.__get_text(path)
            lengths.append(len(text))
        return np.array(lengths)
    
    def __get_sample_from_text(self, text):
        length = min(random.randint(self.padding_limit, self.padding_factor), self.seq_length, len(text)-self.target_length)
        start_idx = random.randint(0, len(text)-length-self.target_length)
        
        seq = self.__padd(text[start_idx : start_idx+length])
        target = text[start_idx+length:start_idx+length+self.target_length]
        
        seq_embed = self.wv[seq]
        target_embed = self.wv[target]
        
        return np.array(seq_embed), np.array(target_embed)
    
    def __padd(self, text):
        if len(text) < self.seq_length:
            padding = ['kot']*(self.seq_length-len(text))
            text = padding + text
        return text

In [3]:
# wv = Word2Vec.load('../../models/word2vec/word2vec').wv

# dataset = TextTrainDataset('../../data/training_prepared/', wv)

# dl = DataLoader(
#     dataset,
#     num_workers=16,
#     batch_size=64,
#     prefetch_factor=64
# )

In [4]:
class LstmTextGenerator(LightningModule):
    
    VALIDATION_PROMPTS = [
        'Pewnego dnia czerwony kapturek szedł przez las z koszyczkiem jedzenia do swojej babci, która mieszkała w lesie. Śledził go jednak zły wilk, który chciał zjeść dziewczynkę.',
        'Za siedmioma górami i za siedmioma rzekami było sobie królestwo, w którym mieszkała księżniczka',
        'Dawno, dawno temu żył pewien chłopiec',
    ]
    
    VALIDATION_TEMPERATURES = [0.01, 0.2, 0.3, 0.5, 0.7]
    
    
    def __init__(
        self,
        
        # files
        vocabulary_path,
        train_file_path,
        
        # architecture
        embedding_dim=100,
        lstm_layers=1,
        lstm_dropout=0,
        lstm_hidden_size=100,
        dropout=0.2,
        bidirectional=False,
        
        # training process
        batch_size=64,
        seq_length=10, 
        target_length=10,
        target_weight_decrease=1.0,
        padding_factor=20,
        padding_limit=3,
        epoch_size=10,
    ):
        super().__init__()
        self.save_hyperparameters()
        
        self.vocabulary = torch.load(self.hparams.vocabulary_path)
        self.vocabulary.append_token('<pad>')
        
        self.embedding = nn.Embedding(
            len(self.vocabulary),
            self.hparams.embedding_dim
        )
        
        self.lstm = nn.LSTM(
            input_size=self.hparams.embedding_dim,
            hidden_size=self.hparams.lstm_hidden_size,
            batch_first=True,
            num_layers=self.hparams.lstm_layers,
            dropout=self.hparams.lstm_dropout,
            bidirectional=self.hparams.bidirectional
        )
        
        self.fc = nn.Linear((2 if self.hparams.bidirectional else 1)*self.hparams.lstm_hidden_size, len(self.vocabulary))
        
        self.dropout = nn.Dropout(self.hparams.dropout)
        
        self.loss = nn.CrossEntropyLoss()
        
    def generate(self, prompt, length=50, temperature=0.5):
        generated = prompt
        prompt = self.__preprocess_prompt(prompt)
        
        for _ in range(length):
            input_tensor = torch.unsqueeze(torch.tensor(prompt, device=self.device), dim=0)
            next_word_logits = self(input_tensor)[0]
            word_idx = self.__get_word_from_logits(next_word_logits, temperature)
            prompt = prompt[1:] + [word_idx]
            
            word = self.vocabulary.lookup_token(word_idx)
            if generated[-1] in '.!?':
                word = word.capitalize()
            if word not in list('.!?,'):
                generated += ' '
            generated += word
        
        return generated
    
    def __get_word_from_logits(self, next_word_logits, temperature=0.5):
        scaled_logits = next_word_logits / temperature
        adjusted_probs = F.softmax(scaled_logits, dim=-1)
        next_word_index = torch.multinomial(adjusted_probs, num_samples=1).item()
        return next_word_index
        
    def forward(self, x):
        out = self.embedding(x)
        out, _ = self.lstm(out)
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        return out
        
    def training_step(self, batch, batch_no):
        texts, targets = batch
        loss = 0
        weight = 1.0
        
        for i in range(self.hparams.target_length):
            predicted = self.forward(texts)
            loss_part = self.loss(predicted, targets[:, i])
            loss += weight * loss_part
            weight *= self.hparams.target_weight_decrease
            
            words_idx = predicted.argmax(dim=-1)
            texts = torch.cat((texts[:, 1:], words_idx.unsqueeze(1)), axis=1)
        
        self.log('train_loss', loss)
        return loss
    
    def on_train_epoch_end(self):
        tensorboard = self.logger.experiment
        
        for no, validation_prompt in enumerate(self.VALIDATION_PROMPTS):
            for temperature in self.VALIDATION_TEMPERATURES:
                text = self.generate(validation_prompt, length=300, temperature=temperature)
                tensorboard.add_text(f'text_{no}_{temperature}', text, global_step=self.current_epoch)
        
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=0.001)
        return optimizer
    
    def train_dataloader(self):
        dataset = TextTrainDataset(
            self.hparams.train_file_path,
            ...,
            seq_length=self.hparams.seq_length,
            target_length=self.hparams.target_length,
            padding_factor=self.hparams.padding_factor,
            padding_limit=self.hparams.padding_limit,
            epoch_size=self.hparams.epoch_size,
        )
        
        return DataLoader(
            dataset=dataset,
            batch_size=self.hparams.batch_size,
            num_workers=12
        )
        
    def __preprocess_prompt(self, prompt):
        tokenized = self.__tokenize(prompt)
        words_idx = self.vocabulary(tokenized)
        words_idx = [idx for idx in words_idx if idx != -1]
        padding = [0]*(max(self.hparams.seq_length-len(prompt), 0))
        prompt = padding + words_idx
        return prompt
    
    def __tokenize(self, text):
        text = text.lower()
        text = re.sub(r'[^a-ząćęłńóśźż.,!?\- ]', ' ', text)
        text = re.sub(r'([,-.!?])', ' \\1 ', text)
        text = [word for word in text.split(' ') if word]
        return text

In [4]:
logger = TensorBoardLogger(
    save_dir='../..',
    name='logs'
)

trainer = Trainer(
    accelerator='cuda',
    max_epochs=-1,
    enable_progress_bar=True,
    logger = logger,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
generator = LstmTextGenerator(
    # files
    train_file_path='../../data/binary_texts/ebooks17k.pickle',
    vocabulary_path='../../models/vocabulary.pth',
    
    # architecture
    lstm_layers=3,
    lstm_dropout=0.2,
    lstm_hidden_size=100,
    dropout=0.2,
    bidirectional=False,
    
    # training
    seq_length=25,
    target_length=1,
    target_weight_decrease=0.7,
    batch_size=128,
    padding_factor=100,
    padding_limit=4,
    epoch_size=2_000_000,
)

In [5]:
# generator = LstmTextGenerator.load_from_checkpoint(
#     '/home/klima7/studies/piat/Story-Generator/logs/version_3/checkpoints/epoch=27-step=110516.ckpt',
#     batch_size=3048,
#     epoch_size=6_000_000,
# )

/home/klima7/studies/piat/Story-Generator/conda/pytorch/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
summary(
    generator,
    input_size=(64, 20),
    col_names=['input_size', 'output_size', 'num_params', 'params_percent'],
    dtypes=[torch.LongTensor],
    device='cpu'
)

/home/klima7/studies/piat/Story-Generator/conda/pytorch/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/klima7/studies/piat/Story-Generator/conda/pytorch/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Param %
LstmTextGenerator                        [64, 20]                  [64, 150001]              --                             --
├─Embedding: 1-1                         [64, 20]                  [64, 20, 100]             15,000,100                 49.62%
├─LSTM: 1-2                              [64, 20, 100]             [64, 20, 100]             80,800                      0.27%
├─Dropout: 1-3                           [64, 20, 100]             [64, 20, 100]             --                             --
├─Linear: 1-4                            [64, 100]                 [64, 150001]              15,150,101                 50.11%
Total params: 30,231,001
Trainable params: 30,231,001
Non-trainable params: 0
Total mult-adds (G): 2.03
Input size (MB): 0.01
Forward/backward pass size (MB): 78.85
Params size (MB): 120.92
Estimated Total Size (MB): 199.78

In [6]:
trainer.fit(generator)

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | vocabulary | Vocab            | 0     
1 | embedding  | Embedding        | 15.0 M
2 | lstm       | LSTM             | 242 K 
3 | fc         | Linear           | 15.2 M
4 | dropout    | Dropout          | 0     
5 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
30.4 M    Trainable params
0         Non-trainable params
30.4 M    Total params
121.570   Total estimated model params size (MB)
/home/klima7/studies/piat/Story-Generator/conda/pytorch/lib/pyth

Epoch 6:  36%|███▋      | 5699/15691 [01:45<03:05, 53.90it/s, v_num=2] 

/home/klima7/studies/piat/Story-Generator/conda/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
generator.generate('dawno, dawno temu, za siedmioma górami i siedmioma', temperature=1)

'dawno, dawno temu, za siedmioma górami i siedmioma zimy, wiadro były balony i w książkach na fachu przystrojone się aż lekko ptak się spotkały przez by węgiel nie złożę wszystkie zwierzęta na stałe zdrowie cicho przepisane znaczy fabryczne ich głośne groszy dzwonek co się wziął wziął dba z, ten dzień miesiąc szybko zaproszę pisać diety pięknie'

In [ ]:
generator.generate('Pewnego słonecznego dnia czerwony kapturek szedł do swojej babci z koszyczkiem', temperature=1)

'Pewnego słonecznego dnia czerwony kapturek szedł do swojej babci z koszyczkiem powrotem. - cóż to jest. kundel aż z dala. - super mały góra, albo zachować ładnie piskiem orzech, autorka l. mróz - cieślik wierszyk z obrazkiem - bajeczki - pręgi, uwaga, sio. ja gotowy - - wnuczek coś złoży! mamo'